In [1]:
from pvtpy.pvt import PVT
import pvtpy.black_oil as bk
from pvtpy.fluids import Gas, InitialConditions
from pvtpy.pvt import PVT
from pvtpy.compositional import Chromatography
from pvtpy.units import Pressure, Temperature
import numpy as np
import pandas as pd
import seaborn as sns

# Create a chromatography composition

In [2]:
composition_dict = {
    'mole_fraction':[0.0194,0.004,0.9598,0.0023,0.0057,0.0062,0.0006,0.0003,0.0001,0.0005,0.001,0.0001],
    'compound':['carbon-dioxide','nitrogen','methane','ethane','propane','isobutane','n-butane','isopentane','n-pentane','n-hexane','n-heptane','n-octane']
}
chr = Chromatography()
chr.from_df(pd.DataFrame(composition_dict), name='compound', mole_fraction='mole_fraction')

print(chr.df())

                acentric_factor        cas  critical_pressure  \
carbon-dioxide           0.2276   124-38-9             1071.0   
nitrogen                 0.0403  7727-37-9              493.1   
methane                  0.0115    74-82-8              666.4   
ethane                   0.0995    74-84-0              706.5   
propane                  0.1523    74-98-6              616.0   
isobutane                0.1770    75-28-5              527.9   
isopentane               0.2275    78-78-4              490.4   
n-hexane                 0.3013   110-54-3              436.9   
n-heptane                0.3495   142-82-5              396.8   
n-octane                 0.3996   111-65-9              360.7   

                critical_temperature formula    id  \
carbon-dioxide                 87.91     CO2  50.0   
nitrogen                     -232.51      N2  56.0   
methane                      -116.67     CH4   1.0   
ethane                         89.92    C2H6   2.0   
propane       

## Estimate some properties

### Apparent Molecular Weigt

The apparent Molecular Weight (ma) is calculated by summing the product of molar fraction and molecular weight of each component in the chromatography

In [3]:
chr.apparent_molecular_weight()

17.23273911738217

### Gas specific gravity

The Gas specific gravity is calculated by diving the **ma** by the specific gravity of the *air*

In [4]:
chr.gas_sg()

0.5950531463184451

### Pseudo critical properties

The Pseudo critical properties are calulated by summing the product of mole fraction and critical properties (pressure and temperature). By default it corrects the properties by Non-hydrocarbon components with the **wichert-aziz** correlation. 

In [5]:
chr.get_pseudo_critical_properties()

CriticalProperties(critical_pressure=Pressure(value=665.8662951011108, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=-110.52626724778492, unit=<TemperatureUnits.farenheit: 'farenheit'>))

In [6]:
chr.get_pseudo_critical_properties(correct=False)

CriticalProperties(critical_pressure=Pressure(value=672.0385169618734, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=352.38010707495243, unit=<TemperatureUnits.rankine: 'rankine'>))

In [7]:
chr.get_pseudo_critical_properties(correct_method='carr_kobayashi_burrows')

CriticalProperties(critical_pressure=Pressure(value=679.9000200140099, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=-109.84368057640353, unit=<TemperatureUnits.farenheit: 'farenheit'>))

### Get the compressibility factor of gas

Estimate the compressibility factor by estimating the critical properties and applying the default correlation method **papay**

In [8]:
chr.get_z(
    pressure=Pressure(value = 1500, unit='psi'), 
    temperature=Temperature(value = 560, unit='farenheit'),
)

,z
pressure,
1500.0,0.994968


In [9]:
p_range = np.linspace(1000,5000,10)
chr.get_z(pressure=Pressure(value=p_range), temperature=Temperature(value=180, unit='farenheit'))

,z
pressure,
1000.000000,0.935595
1444.444444,0.919677
1888.888889,0.911579
2333.333333,0.911301
2777.777778,0.918843
3222.222222,0.934204
3666.666667,0.957385
4111.111111,0.988385
4555.555556,1.027205


### Get the gas density in lb/ft3

Estimate the gas density by estimating the **ma**, the **z** factor and finnaly applying the gas equation of state for **real gases**

In [10]:
chr.get_rhog(
    pressure=Pressure(value=300, unit='psi'),
    temperature=Temperature(value=180, unit='farenheit')
)

,rhog
pressure,
300.0,0.771326


In [11]:
chr.get_rhog(
    pressure=Pressure(value=1300, unit='psi'),
    temperature=Temperature(value=180, unit='farenheit'), 
    rhog_method='ideal_gas')

,rhog
pressure,
1300.0,3.263938


In [12]:
chr.get_rhog(
    pressure=Pressure(value=np.linspace(1000,5000,10)),
    temperature=Temperature(value=180, unit='farenheit'), 
    rhog_method='real_gas'
)

,rhog
pressure,
1000.000000,2.683557
1444.444444,3.943338
1888.888889,5.202481
2333.333333,6.428556
2777.777778,7.590229
3222.222222,8.659890
3666.666667,9.615758
4111.111111,10.443151
4555.555556,11.134805


### Estimate the Specific volume of Gas. 

Get the specific volume by estimate the inverse of the density

In [13]:
chr.get_sv(
    pressure=Pressure(value=np.linspace(1000,5000,10)),
    temperature=Temperature(value=180, unit='farenheit'), 
    rhog_method='real_gas'
)

pressure
1000.000000    0.372640
1444.444444    0.253592
1888.888889    0.192216
2333.333333    0.155556
2777.777778    0.131748
3222.222222    0.115475
3666.666667    0.103996
4111.111111    0.095757
4555.555556    0.089808
5000.000000    0.085541
Name: sv, dtype: float64

In [14]:
chr.get_sv(    pressure=Pressure(value=np.linspace(1000,5000,10)),
    temperature=Temperature(value=180, unit='farenheit'), 
    rhog_method='real_gas')

pressure
1000.000000    0.372640
1444.444444    0.253592
1888.888889    0.192216
2333.333333    0.155556
2777.777778    0.131748
3222.222222    0.115475
3666.666667    0.103996
4111.111111    0.095757
4555.555556    0.089808
5000.000000    0.085541
Name: sv, dtype: float64

In [15]:
chr.gas_sg()

0.5950531463184451

# Create the Gas object

In [16]:
init = InitialConditions(pressure=Pressure(value=3600), temperature=Temperature(value=160))

In [17]:
gas1 = Gas(
    initial_conditions=init,sg=0.59,
    chromatography=chr
)


In [18]:
gas1.pseudo_critical_properties()

CriticalProperties(critical_pressure=Pressure(value=665.8662951011108, unit=<PressureUnits.psi: 'psi'>), critical_temperature=Temperature(value=-110.52626724778492, unit=<TemperatureUnits.farenheit: 'farenheit'>))

In [23]:
gas1.pvt_from_correlations()
gas1.pvt.df()

,z,rhog,bg,mug,cg
pressure,,,,,
20.000000,0.998092,0.051934,0.155550,0.013007,0.050000
282.105263,0.974719,0.750112,0.010770,0.013181,0.003545
544.210526,0.954374,1.477890,0.005466,0.013451,0.001838
806.315789,0.937058,2.230143,0.003622,0.013794,0.001240
1068.421053,0.922771,3.000839,0.002692,0.014201,0.000936
1330.526316,0.911512,3.783165,0.002135,0.014668,0.000752
1592.631579,0.903281,4.569686,0.001768,0.015189,0.000628
1854.736842,0.898079,5.352561,0.001509,0.015761,0.000539
2116.842105,0.895906,6.123788,0.001319,0.016375,0.000472
